In [1]:
import numpy as np
import mySDC

In [2]:
# equidistant nodes, phi functions
t0 = 1.0
u0 = 1.0 + 0.j
steps = 5

timestep = 0.1
t = np.linspace(t0, t0 + timestep*(steps-1), steps)

dt = t[1:] - t[:-1]
N = steps-2

L = 1.
w, phi0, phi1 = mySDC.generate_weights(N, t[0:N+2], dt[0:N+1], L)
# should be 0
assert np.all(phi0 - np.exp(dt[0:N+2] * L) < 1e-16), "oops"
assert np.all(phi1 - (np.exp(dt[0:N+2] * L) - 1.) / (dt[0:N+2] * L) < 1e-16), "oops"

# if l1=0 (N=I) then integral should produce substeps
L = 0.
w, phi0, phi1 = mySDC.generate_weights(N, t[0:N+2], dt[0:N+1], L)
assert np.all(np.sum(w, axis=1) - dt < 1e-16), "oops"

In [4]:
# on equidistant
L1 = 0.
L2 = 5.
print(t, dt)

w, phi0, phi1 = mySDC.generate_weights(N, t[0:N+2], dt[0:N+1], L1)

# N = l2*t
N_ = t[0:N+2] * L2

print("     ", np.dot(w,  N_).real)
print("true:", L2 / 2 * (t[1:]**2 - t[:-1]**2))
    #assert np.isclose([diff], [0.0]), "ooops"
    
# N = l2*t**2
N_ = t[0:N+2]**2 * L2

print("     ", np.dot(w,  N_).real)
print("true:", L2 / 3 * (t[1:]**3 - t[:-1]**3))
    #assert np.isclose([diff], [0.0]), "ooops"
    
# N = l2*exp(t)
N_ = np.exp(t[0:N+2]) * L2
print("     ", np.dot(w,  N_).real)
print("true:", L2 * (np.exp(t[1:]) - np.exp(t[:-1])))

[1.  1.1 1.2 1.3 1.4] [0.1 0.1 0.1 0.1]
      [0.525 0.575 0.625 0.675]
true: [0.525 0.575 0.625 0.675]
      [0.55166667 0.66166667 0.78166667 0.91166667]
true: [0.55166667 0.66166667 0.78166667 0.91166667]
      [1.42942067 1.57975462 1.7458986  1.92951682]
true: [1.42942098 1.57975449 1.74589872 1.9295165 ]


In [8]:
# Chebyshev
t0 = 0.0
u0 = 1.0 + 0.j
steps = 5
N = steps-2

timestep = 1.
t = np.linspace(t0, t0 + timestep*(steps-1), steps)

Ti = (N+1) * (steps-1) + 1
tau = np.zeros((Ti))
taui, _ = np.polynomial.chebyshev.chebgauss(N)
    
for i in range(steps-1):
    tau[(N+1)*i] = t[i]
    tau[(N+1)*i+1:(N+1)*i+(N+1)] = taui[::-1] / 2 * timestep + timestep / 2 + t[i]
tau[-1] = t[steps-1]    
dtau = tau[1:] - tau[:-1]

L = 1.
w, phi0, phi1 = mySDC.generate_weights(N, tau[0:N+2], dtau[0:N+1], L)

# should be 0
assert np.all(phi0 - np.exp(dtau[0:N+1] * L) < 1e-6), "oops"
assert np.all(phi1 - (np.exp(dtau[0:N+1] * L) - 1.) / (dtau[0:N+1] * L) < 1e-6), "oops"

# if l1=0 (N=I) then integral should produce substeps
L = 0.
w, phi0, phi1 = mySDC.generate_weights(N, tau[0:N+2], dtau[0:N+1], L)
assert np.all(np.sum(w, axis=1) - dtau[0:N+1] < 1e-6), "oops"

In [14]:
L1 = 0.
L2 = 3.
print(tau[0:N+2], dtau[0:N+1])

w, phi0, phi1 = mySDC.generate_weights(N, tau[0:N+2], dtau[0:N+1], L1)

# N = l2
N_ = tau[0:N+2] * 0. + L2
print("     ", np.dot(w,  N_).real)
print("true:", L2 * dtau[:N+1])

# N = l2*t
N_ = tau[0:N+2] * L2
print("     ", np.dot(w,  N_).real)
print("true:", L2 / 2 * (tau[1:N+2]**2 - tau[:N+1]**2))

# N = l2*t**2
N_ = tau[0:N+2]**2 * L2
print("     ", np.dot(w,  N_).real)
print("true:", L2 / 3 * (tau[1:N+2]**3 - tau[:N+1]**3))

# N = l2*exp(t)
N_ = np.exp(tau[0:N+2]) * L2
print("     ", np.dot(w,  N_).real)
print("true:", L2 * (np.exp(tau[1:N+2]) - np.exp(tau[:N+1])))

[0.        0.0669873 0.5       0.9330127 1.       ] [0.0669873 0.4330127 0.4330127 0.0669873]
      [0.20096189 1.29903811 1.29903811 0.20096189]
true: [0.20096189 1.29903811 1.29903811 0.20096189]
      [0.00673095 0.36826905 0.93076905 0.19423095]
true: [0.00673095 0.36826905 0.93076905 0.19423095]
      [3.00591976e-04 1.24699408e-01 6.87199408e-01 1.87800592e-01]
true: [3.00591976e-04 1.24699408e-01 6.87199408e-01 1.87800592e-01]
      [0.2078449  1.73838446 2.68023432 0.52837717]
true: [0.20784569 1.73831812 2.68030542 0.52837625]
